In [ ]:
import imageio
import numpy as np
import pandas as pd
from skimage import transform
from IPython import display
from matplotlib import pyplot as plt
from src import camvid
from src import tiramisu
from src.utils import heatmap
from src.wrappers import MonteCarlo
%matplotlib inline
plt.style.use('dark_background')

In [ ]:
mapping = pd.read_table('11_class.txt', sep=r'\s+', names=['og', 'new'], index_col='og')['new'].to_dict()
camvid11 = camvid.CamVid(mapping=mapping)

In [ ]:
video_file = 'camvid/0005VD.mp4'
video_file

In [ ]:
out_file = video_file.split('.')[0] + '_pred.mp4'
out_file

In [ ]:
reader = imageio.get_reader(video_file)

In [ ]:
og_shape = reader.get_data(0).shape
og_shape

In [ ]:
image_shape = (352, 480, 3)
image_shape

In [ ]:
# build the model for the image shape and number of labels
model = tiramisu.build_epistemic_tiramisu(image_shape, camvid11.n)
model.layers[1].load_weights('models/Tiramisu103-CamVid11-fine.h5')

In [ ]:
def predict(frame) -> tuple:
    """
    Return post-processed predictions for the given frame.

    Args:
        frame: the frame to predict

    Returns:
        a tuple of for NumPy tensors with RGB data:
        - the unmapped RGB predicted mean values from the model
        - the meatmap RGB values of the model variance

    """
    # predict mean values and variance
    X = frame[None, ...]
    u, s2 = model.predict(X)
    # normalize the uncertainty
    s2 = plt.Normalize()(s2)
    # return X values, unmapped u values, and heatmap of s2
    return camvid11.unmap(u)[0], heatmap(s2, 'afmhot')[0]

In [ ]:
writer = imageio.get_writer(out_file)


for frame in reader:
    # resize the image to the acceptable size for the
    # model
    X = transform.resize(frame, image_shape,
        anti_aliasing=False,
        mode='symmetric',
        clip=False,
        preserve_range=True,
    )
    # predict mean and variance of the frame
    u, s2 = predict(X)
    h, w, c = image_shape
    # convert the three images into a singular image
    image = np.zeros((h, w * 3, c), dtype='uint8')
    image[:, 0:w, :] = X
    image[:, w:2*w, :] = u
    image[:, 2*w:3*w, :] = s2
    # save the image to the stream
    writer.append_data(image)


writer.close()